### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# check unqiue latest feature id
model_data['latest_feature_id'].nunique()

478328

In [4]:
model_data['unique_reference_number'].nunique()

478327

### Drop Null target records

In [5]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [6]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [7]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [8]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [9]:
features_list = ['Num_Enq_all_loans_last_6M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_9M',
'num_CREDIT CARD_enq_last_3M',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
'total_credit_all_open',
'num_PL_LE_1_Lac_opened_in_last_3M',
    
# Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

    
#ID Features

'aggs907',
'dm216s',
'dm206s',
'balmag01',

'month_year',
'60_in_15mob']

In [10]:
len(features_list)

22

In [11]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [12]:
oot_df = oot_df1[features_list]

In [13]:
model_df.shape, oot_df.shape

((101984, 22), (17343, 22))

In [14]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [15]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [16]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [17]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [18]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3028559/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [19]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3028559/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [20]:
X_oot.shape,y_oot.shape

((17343, 20), (17343,))

### Split Model dataset into Train and Test

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [22]:
X_train.shape, X_test.shape

((61190, 20), (40794, 20))

In [23]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 20) (12238, 20) (40794, 20)


In [24]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [25]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7,
    'alpha': 4.5,
    'gamma': 4.8,
    'scale_pos_weight':4.0,
    'random_state': 42
    
}



evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.62276	eval-auc:0.62207
[10]	train-auc:0.64589	eval-auc:0.63935
[20]	train-auc:0.64890	eval-auc:0.64246
[30]	train-auc:0.64991	eval-auc:0.64527
[40]	train-auc:0.65198	eval-auc:0.64610
[50]	train-auc:0.65357	eval-auc:0.64808
[60]	train-auc:0.65403	eval-auc:0.64889
[70]	train-auc:0.65522	eval-auc:0.64996
[80]	train-auc:0.65596	eval-auc:0.65102
[90]	train-auc:0.65734	eval-auc:0.65189
[100]	train-auc:0.65808	eval-auc:0.65297
[110]	train-auc:0.65907	eval-auc:0.65317
[120]	train-auc:0.65946	eval-auc:0.65336
[130]	train-auc:0.66019	eval-auc:0.65338
[140]	train-auc:0.66083	eval-auc:0.65363
[150]	train-auc:0.66150	eval-auc:0.65400
[160]	train-auc:0.66225	eval-auc:0.65445
[170]	train-auc:0.66288	eval-auc:0.65513
[180]	train-auc:0.66364	eval-auc:0.65574
[190]	train-auc:0.66436	eval-auc:0.65603
[200]	train-auc:0.66512	eval-auc:0.65682
[210]	train-auc:0.66577	eval-auc:0.65738
[220]	train-auc:0.66645	eval-auc:0.65805
[230]	train-auc:0.66699	eval-auc:0.65830
[240]	train-auc:0.66783	eva

In [26]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.37, KS: 0.27
Eval AUC: 0.67, Gini: 0.34, KS: 0.25
Test AUC: 0.67, Gini: 0.34, KS: 0.24
OOT AUC: 0.68, Gini: 0.36, KS: 0.27


In [27]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.69, Gini: 0.3741, KS: 0.2709
Test AUC: 0.67, Gini: 0.3353, KS: 0.2372
OOT AUC: 0.68, Gini: 0.3625, KS: 0.2675


In [28]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
9                   num_PL_LE_1_Lac_opened_in_last_3M       198.0
14                          Num_Enq_non_BANK_last_12M       197.0
5                highest_sanction_amount_credit_cards       160.0
0                           Num_Enq_all_loans_last_6M       159.0
16                                            aggs907       158.0
8                               total_credit_all_open       142.0
7             Utilization_CREDIT CARD_opened_last_36M       140.0
19                                           balmag01       135.0
12                       percentage_utilization_gt_75       130.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...       122.0
18                                             dm206s       119.0
10                         percentage_ever_delinquent       114.0
2                       num_CONSUMER LOAN_enq_last_9M       112.0
6                          Age_of_oldest_open_account       109.0
13        

In [29]:
feature_importance_df

Feature  Importance
9                   num_PL_LE_1_Lac_opened_in_last_3M       198.0
14                          Num_Enq_non_BANK_last_12M       197.0
5                highest_sanction_amount_credit_cards       160.0
0                           Num_Enq_all_loans_last_6M       159.0
16                                            aggs907       158.0
8                               total_credit_all_open       142.0
7             Utilization_CREDIT CARD_opened_last_36M       140.0
19                                           balmag01       135.0
12                       percentage_utilization_gt_75       130.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...       122.0
18                                             dm206s       119.0
10                         percentage_ever_delinquent       114.0
2                       num_CONSUMER LOAN_enq_last_9M       112.0
6                          Age_of_oldest_open_account       109.0
13                           No of enq in past 90days        96.0
11   months_since_most_recent_delinquency_installment        86.0
15                     Num_of_installment_loan_opened        86.0
17                                             dm216s        71.0
3                         num_CREDIT CARD_enq_last_3M        52.0
1                      num_PERSONAL LOAN_enq_last_36M        12.0

In [30]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.274510         0.558371        0.838835        0.468128   4896   
9          0.165066         0.426997        0.468122        0.395216   4895   
8          0.141164         0.370909        0.395189        0.348952   4895   
7          0.105822         0.330919        0.348932        0.314126   4895   
6          0.100306         0.298502        0.314126        0.283595   4895   
5          0.083759         0.270584        0.283593        0.257726   4895   
4          0.068437         0.245638        0.257724        0.233917   4895   
3          0.061900         0.222129        0.233913        0.209687   4895   
2          0.044944         0.194706        0.209686        0.177709   4895   
1          0.033088         0.154022        0.177704        0.098137   4896   

        label_good  label_bad  
Decile                         
10            3552       1344  
9             4087        808  
8             4204        691  
7             4377        518  
6             4404        491  
5             4485        410  
4             4560        335  
3             4592        303  
2             4675        220  
1             4734        162

In [31]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.261765         0.556206        0.832794        0.466527   4080   
9          0.155921         0.427344        0.466473        0.394891   4079   
8          0.142682         0.370710        0.394887        0.349327   4079   
7          0.113235         0.330267        0.349326        0.313067   4080   
6          0.101250         0.297981        0.313064        0.283297   4079   
5          0.095857         0.270235        0.283286        0.257145   4079   
4          0.079412         0.245686        0.257145        0.234129   4080   
3          0.064231         0.222057        0.234127        0.209246   4079   
2          0.055161         0.194652        0.209243        0.178118   4079   
1          0.036765         0.154573        0.178116        0.102432   4080   

        label_good  label_bad  
Decile                         
10            3012       1068  
9             3443        636  
8             3497        582  
7             3618        462  
6             3666        413  
5             3688        391  
4             3756        324  
3             3817        262  
2             3854        225  
1             3930        150

In [32]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.251297         0.568689        0.814109        0.474751   1735   
9          0.167820         0.429293        0.474700        0.394558   1734   
8          0.144175         0.368601        0.394519        0.345467   1734   
7          0.104960         0.326751        0.345415        0.309039   1734   
6          0.095732         0.293757        0.309037        0.279565   1734   
5          0.092795         0.266378        0.279561        0.253823   1735   
4          0.066321         0.242509        0.253775        0.231249   1734   
3          0.053633         0.219398        0.231234        0.207417   1734   
2          0.050173         0.193048        0.207372        0.175711   1734   
1          0.032853         0.152080        0.175683        0.102072   1735   

        label_good  label_bad  
Decile                         
10            1299        436  
9             1443        291  
8             1484        250  
7             1552        182  
6             1568        166  
5             1574        161  
4             1619        115  
3             1641         93  
2             1647         87  
1             1678         57

In [33]:
#Save the Model
xgb_model.save_model('LeoPayu_BI_60_in_15mob.json')
print('Model Saved')

Model Saved


In [34]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]


In [35]:
Train.to_csv('LeoPayu_BureauID_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_BureauID_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_BureauID_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_BureauID_Oot_Pred.csv',index=False)
